### QUESTION 1: Factors that impact salary

To predict salary you will be building either a classification or regression model, using features like the location, title, and summary of the job. If framing this as a regression problem, you will be estimating the listed salary amounts. You may instead choose to frame this as a classification problem, in which case you will create labels from these salaries (high vs. low salary, for example) according to thresholds (such as median salary).

You have learned a variety of new skills and models that may be useful for this problem:
- NLP
- Unsupervised learning and dimensionality reduction techniques (PCA, clustering)
- Ensemble methods and decision tree models
- SVM models

Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to able to extrapolate or predict the expected salaries for these listings.

<b>** abandoned because no salary data and already had indeed and careersfuture scrapes.**

In [76]:
from bs4 import BeautifulSoup
import urllib
import urllib.parse
from time import sleep
import pandas as pd
from selenium import webdriver
import datetime
import getpass
from selenium.webdriver.common import action_chains, keys

In [77]:
import os

# # Create a driver called "driver."
# driver = webdriver.Chrome(executable_path="../chromedriver/chromedriver")

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")
options.binary_location = "/usr/bin/chromium"
driver = webdriver.Chrome("C:/Users/schen24/Anaconda3/Library/chromedriver.exe")


In [78]:
# Set the URL we want to visit.
searchstr = "Data Engineer"
plus = searchstr.replace(" ", '+')
location = "Singapore"
page = 0
jtype = ""  # "fulltime", "permanent", "contract","internship", "temporary", "parttime"
baseurl = "https://www.jobstreet.com.sg"
url = baseurl + "/en/job-search/job-vacancy.php?ojs=2&key={}".format(plus)
print(url)


https://www.jobstreet.com.sg/en/job-search/job-vacancy.php?ojs=2&key=Data+Engineer


In [79]:
driver.get(baseurl)

In [80]:
# Login to Jobstreet 
search_job_button = driver.find_element_by_id('header-login-button')
search_job_button.click()
# Rest before interacting with the login bar
sleep(1)
# Enter username
username = input('username:')
username_bar = driver.find_element_by_name('login_id')
username_bar.clear()
username_bar.send_keys(username)
# Enter password
password = getpass.getpass('password:')
pw_bar = driver.find_element_by_name('password')
pw_bar.clear()
pw_bar.send_keys(password)
# Log in 
login_button = driver.find_element_by_name('btnSignIn')
login_button.click()


username:kpryde
password:········


In [81]:
# Wait one second.
sleep(1)

In [83]:
driver.get(url)

In [84]:
# Beautiful Soup it!
soup = BeautifulSoup(driver.page_source, 'lxml')

In [85]:
soup

<!DOCTYPE html>
<html class=" cus-sg js flexbox canvas canvastext webgl no-touch geolocation postmessage websqldatabase indexeddb hashchange history draganddrop websockets rgba hsla multiplebgs backgroundsize borderimage borderradius boxshadow textshadow opacity cssanimations csscolumns cssgradients cssreflections csstransforms csstransforms3d csstransitions fontface no-generatedcontent video audio localstorage sessionstorage webworkers applicationcache svg inlinesvg smil svgclippaths" style="" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
<title>Data Engineer Jobs in Singapore, Job Vacancies | JobStreet.com.sg </title>
<meta content="New Data Engineer Jobs in Singapore available today on JobStreet - Quality Candidates, Quality Employers, 59767 vacancies " name="description"/>
<meta content="Data Engineer, " name="keywo

**write loop to extract all links**

In [86]:
soup.find_all('a', {"class":"position-title-links"})

[]

In [70]:
links = []
for i in range(1,10):
    try:
        hits = soup.find_all('a', {"class":"position-title-links"})
        #print(hits)
        elems = driver.find_elements_by_xpath("//a[@href]")
        for elem in elems:
        #print(elem.get_attribute("href"))
            if 'job' in elem.get_attribute("href"):
                links.append(elem.get_attribute("href"))

        nextpg = baseurl + "/search?search={}&sortBy=new_posting_date&page={}".format(plus, i)
        driver.get(nextpg)
        sleep(2)
        soup = BeautifulSoup(driver.page_source, 'lxml')
    except:
        continue

In [71]:
# let's check out number of items in links
len(links)

48

In [72]:
links[:5]

['https://www.mycareersfuture.sg/job/data-visualisation-engineer-8a1a2ec41c135c42eabd062ad3c83471',
 'https://www.mycareersfuture.sg/job/research-assistant-national-university-singapore-61b2e7a6347ebb9e1f4da0a3ed544639',
 'https://www.mycareersfuture.sg/job/required-data-centre-engineer-12-months-contract-path-infotech-7616f787bea6fc4c9d046896a7879562',
 'https://www.mycareersfuture.sg/job/data-center-engineer-techcom-solutions-consultancy-de3140d8875464b26482d29c4a96ff78',
 'https://www.mycareersfuture.sg/job/data-engineer-abakus-ec80cf4678d1bf50b66a251f06283161']

In [73]:
# grey out after first run if extracting data in batches
datadict = []

###  now let's write a loop to extract all the info we want and stick it into pandas

In [74]:

for i in links:
    try:
        driver.get(i)
        sleep(1)
    except:
        continue

    #reinitialize soup object to new page
    soup2 = BeautifulSoup(driver.page_source, 'lxml')
    
    # extracting info
    entry ={}
    #entry['jobid'] = 'jobid'
    entry['url'] = i
    entry['searchstring'] = searchstr
    entry['location'] = location
    try:
        entry['jobtitle'] = soup2.find('h1', {"id":"job_title"}).text
    except:
        entry['jobtitle'] = None
    try:
        entry['advertiser'] = soup2.find('p', {"name":"company"}).text
    except:
        entry['advertiser'] = None
    try:
        h1 = soup2.find('div', {"id":"job_description"}).text
        h2 = soup2.find('div', {"id":"requirements"}).text
        entry['jd'] = h1 +'\n\n'+ h2
    except:
        entry['jd'] = None
    try:
        entry['ojoblink'] = i
    except:
        entry['ojoblink'] = None
    try:
        h = soup2.find('span', {"id":"last_posted_date"}).text
        dt = h.split(' ')[1:4]
        dt = ' '.join(dt)
        daysbet = (datetime.datetime.today() - datetime.datetime.strptime(dt, '%d %b %Y')).days
        entry['elapsed'] = daysbet
    except:
        entry['elapsed'] = None
    try:
        entry['source'] = soup2.find('p', {"name":"company"}).text
    except:
        entry['source'] = None
    try:
        h = soup2.find('span', {"class":"salary_range dib f2-5 fw6 black-80"}).text
        entry['salary'] = h.replace('to', ' ') if 'to' in h else h
    except:
        entry['salary'] = None
    try:
        entry['jobtype'] = soup2.find('p', {"id":"employment_type"}).text
    except:
        entry['jobtype'] = None
    datadict.append(entry)
    

In [75]:
driver.close()

In [76]:
df = pd.DataFrame(datadict)
df.head()

,advertiser,elapsed,jd,jobtitle,jobtype,location,ojoblink,salary,searchstring,source,url
0,Company Undisclosed,9.0,Roles & ResponsibilitiesIf you have a passion ...,IT Data Visualisation Engineer,Full Time,Singapore,https://www.mycareersfuture.sg/job/data-visual...,"$7,000 $12,000",Data Engineer,Company Undisclosed,https://www.mycareersfuture.sg/job/data-visual...
1,NATIONAL UNIVERSITY OF SINGAPORE,9.0,Roles & ResponsibilitiesWe have multiple postd...,Research Assistant,Contract,Singapore,https://www.mycareersfuture.sg/job/research-as...,"$36,000 $60,000",Data Engineer,NATIONAL UNIVERSITY OF SINGAPORE,https://www.mycareersfuture.sg/job/research-as...
2,PATH INFOTECH PTE. LTD.,10.0,Roles & Responsibilities5 Years working experi...,Required Data Centre Engineer-12 Months Contract,Contract,Singapore,https://www.mycareersfuture.sg/job/required-da...,"$3,000 $3,500",Data Engineer,PATH INFOTECH PTE. LTD.,https://www.mycareersfuture.sg/job/required-da...
3,TECHCOM SOLUTIONS & CONSULTANCY PTE. LTD.,10.0,Roles & Responsibilities 4+ years working expe...,Data Center Engineer,"Contract, Full Time",Singapore,https://www.mycareersfuture.sg/job/data-center...,"$5,000 $6,000",Data Engineer,TECHCOM SOLUTIONS & CONSULTANCY PTE. LTD.,https://www.mycareersfuture.sg/job/data-center...
4,None,NaN,None,None,None,Singapore,https://www.mycareersfuture.sg/job/data-engine...,None,Data Engineer,None,https://www.mycareersfuture.sg/job/data-engine...


In [77]:
df.shape

(48, 11)

In [78]:
df.to_pickle("./JS{}.pkl".format(searchstr))